In [16]:
import cv2
import numpy as np
import pandas as pd

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

In [17]:
# Load the Fer2013 dataset
data = pd.read_csv('fer2013.csv')

data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [18]:
# Balance the dataset by oversampling the minority classes
data_balanced = data[data.emotion != 3]
data_balanced = pd.concat([data_balanced, data[data.emotion == 3].sample(n=len(data_balanced), replace=True)])

In [19]:
# Split the dataset into training, validation, and test sets
train_data = data_balanced[data_balanced.Usage == 'Training'] # Split the dataset into training, validation, and test sets
validation_data = data_balanced[data_balanced.Usage == 'PrivateTest']
test_data = data_balanced[data_balanced.Usage == 'PublicTest']


In [20]:
# Extract the pixel values and labels from the dataset
train_pixels = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in train_data.pixels])
validation_pixels = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in validation_data.pixels])
test_pixels = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in test_data.pixels])
train_labels = to_categorical(train_data.emotion.values)
validation_labels = to_categorical(validation_data.emotion.values)
test_labels = to_categorical(test_data.emotion.values)


In [21]:
# Reshape the pixel arrays to 48x48x1
train_pixels = train_pixels.reshape(-1, 48, 48, 1)
validation_pixels = validation_pixels.reshape(-1, 48, 48, 1)
test_pixels = test_pixels.reshape(-1, 48, 48, 1)

In [22]:
# Normalize the pixel values
train_pixels = train_pixels / 255.0
validation_pixels = validation_pixels / 255.0
test_pixels = test_pixels / 255.0

In [23]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D())
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))


In [24]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.save('emotion_model.h5')